# Store Traffic Monitor

Store traffic monitor is an advanced use-case example, which builds on the two previous examples (<a href="../flaw-detector-python/flawdetector-demo.ipynb">flaw detector</a> and <a href="../object-detection-python/object_detection_demo.ipynb">object detector</a>) and shows how a user can perform object detection and inference on multiple videos simultaneously. In this use case, the application monitors the activity of people inside and outside an imaginary store as well as keeps track of product inventory using a pre-trained neural network for detection. This application is capable of detecting objects on any number of screens as long as sufficient compute power is available.

This demo assumes you have already tried the <a href="../object-detection-python/object_detection_demo.ipynb">object detector</a> sample. If you have not done so already, we recommend going through that sample first to understand the workflow for developing a deep learning application using OpenVINO. 

More information on using the store traffic monitor, as well as the original source code, can be found on GitHub:

https://github.com/intel-iot-devkit/store-traffic-monitor-python

Note that the code in this notebook has been slightly modified from the version on GitHub in order to work in the IoT DevCloud environment.

## Overview of How it works?

The counter uses the inference engine included in the OpenVINO™ toolkit. A trained neural network detects objects within a designated area and displays a green bounding box over them. This reference implementation identifies multiple intruding objects entering the frame and identifies their class, count, and time entered. 

At start-up, the sample application reads a configuration file and loads a network and images from the video input to the Inference Engine (IE) plugin. At runtime, computations are offloaded to a the hardware accelerator (Intel® Core CPU, Intel® HD Graphics GPU, Intel® Xeon CPU, Intel® Movidius™ and/or Neural Compute Stick). Once the inference is completed, the output videos are appropriately rendered and stored in the /results directory, which can then be viewed within the Jupyter Notebook instance

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed on edge compute nodes with various target compute devices
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes
* Demonstrate the Async API in action

## Step 0: Setup

Import dependencies for the notebook by running the following cell

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

## Step 1: Advanced OpenVINO

The complete listing of source code for this example is in <a href="store_traffic_monitor.py">store_traffic_monitor.py</a>.
This application takes multiple video streams as inputs. The application is designed to work in both sync and async mode. In sync mode, the processing is done by the compute device by taking one frame at a time, whereas in async mode, the device takes multiple frames and processes them in parallel.


**Configure the Application**

All the configurations are written to store-traffic-monitor-python/conf_fp*.txt. We have two configuration files: conf_fp16.txt and conf_fp32.txt  

- **conf_fp16.txt**: use if you want the inference to be run on Intel® Movidius™ Neural Compute Stick as it offers native FP16 precision.
- **conf_fp32.txt**: use if you want the inference to be run on Intel® Core CPU, Intel® HD Graphics GPU and Intel® Xeon CPU.

1st line: path/to/model.xml -> 
This is the path to the model topology in the local file system.
The model topology file is the .xml file that the model optimizer produces, containing the IR of the model's topology.

2nd line: path/to/model.bin ->
This is the path to the model weights in the local file system.
The model weights file is the .bin file that the model optimizer produces, containing the IR of the model's weights.

3rd line: path/to/labels ->
This is a path to the labels file in the local file system.
The labels file is a text file containing all the classes/labels that the model can recognize, in the order that it was trained to recognize them (one class per line). All detection models work with integer labels and not string labels (e.g. for the mobilenet-ssd model, the **number 15 represents the class "person"**).
For the mobilenet-ssd model, we provide the class file labels.txt in the **"/data/reference-sample-data/store-traffic-monitor-python"** folder.

**Note** : store_traffic_monitor application is used to detect a person and a bottle in a frame. For the mobilenet-ssd model used in this example, the line number 5 and  line number 15 in the labels.txt file correspond to bottle and person, respectively. If you are using a different model, then the items in labels.txt file would need to be adjusted appropriately.

Each of the following lines contain the path/to/video followed by the label to be detected on that video, e.g.:

**/data/reference-sample-data/store-traffic-monitor-python/people-detection.mp4** person

The path/to/video is the path, in the local file system, to a video that is used as input. The labels used must coincide with the labels from the labels file.

**Models to Use ?**

The application works with any object-detection model, provided it has the same input and output format of the mobilenet-ssd model.
The model can be any object detection model:

* that is provided by OpenVINO™ toolkit.
These can be found in the openvino/deployment_tools/intel_models folder.
* downloaded using the model downloader.
These can be found in the openvino/deployment_tools/model_downloader/object_detection folder.
* created by the user

For first-time use, we recommend using the mobilenet-ssd model provided as a part of this application. The models can be found under the folder "/data/reference-sample-data/models/mobilenet-ssd"

**Videos to Use ?**

The application works with any input video. Sample videos for object detection are provided here.

For first-use, we recommend using the people-detection, one-by-one-person-detection, bottle-detection videos.
E.g.:

/data/reference-sample-data/store-traffic-monitor-python/people-detection.mp4 person<br>
/data/reference-sample-data/store-traffic-monitor-python/one-by-one-person-detection.mp4 person<br>
/data/reference-sample-data/store-traffic-monitor-python/bottle-detection.mp4 bottle

For the sake of simplicity, the videos are copied into /data/reference-sample-data/store-traffic-monitor-python/ folder

**Command line arguments options and how they are interpreted in the application source code**

```python
** python3 store_traffic_monitor.py -d CPU -c conf_fp32.txt -o results **
```

#### The description of the arguments used in the argument parser is the command line executable equivalent.

 *  -o  location where the output file with inference results needs to be stored. (results/core or results/xeon or results/gpu or results/myriad)

 *  -d Type of Hardware Acceleration (CPU or GPU or MYRIAD)
 *  -c configuration file to use ("conf_fp32.txt" for Intel® Core CPU, Intel® HD Graphics GPU and Intel® Xeon CPU. "conf_fp16.txt" for Intel® Movidius™ Neural Compute Stick)

### 1.1 Choosing Device
First, we must select the device used for inferencing. This is done by loading the appropriate plugin to initialize the specified device (CPU, GPU, MYRIAD etc) and load the extensions library (if specified) provided in the extension/ folder for the device.

The plugin class is IEPlugin and can be constructed as follows:
```python
    # Parses the configuration file "conf.txt" and reads model_xml, model_bin, labels_file, videoCaps
    parse_conf_file()
    # Plugin initialization for specified device and load extensions library
    print("Initializing plugin for {} device...".format(TARGET_DEVICE))
    plugin = IEPlugin(device=TARGET_DEVICE)
    if 'CPU' in TARGET_DEVICE:
        plugin.add_cpu_extension(CPU_EXTENSION)
```

**Note**: Currently, three types of plugins are supported: CPU, GPU, and MYRIAD. The CPU plugin may require additional extensions to improve performance. Use add_cpu_extension function to load these additional extensions.

### 1.2 Read the IR (Intermediate Representation) model

Intel Model Optimizer creates Intermediate Representation (IR) models that are optimized for different Intel hardware.
We can import these optimized models (weights) into our neural network using **`IENetwork`**. 
```python
    net = IENetwork.from_ir(model=model_xml, weights=model_bin)
    assert len(net.inputs.keys()) == 1, "Sample supports only single input topologies"
    assert len(net.outputs) == 1, "Sample supports only single output topologies"
    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
```

### 1.3 Load the network into the plugin

Once we have the plugin and the network, we can load the IR into the plugin using **`plugin.load`**.

```python
    # Loading IR model to the plugin.
    exec_net = plugin.load(network=net, num_requests=2)
    # Read and pre-process input image
    n, c, h, w = net.inputs[input_blob]
    del net

```

### 1.4 Start video capture using OpenCV 

Now we are ready to capture the frames from the video sample using **OpenCV VideoCapture** API.
Upon getting the frame we are ready to perform inference.

```python
cap = cv2.VideoCapture(input_stream)
```


### 1.5 Prepare the Model

Run the cells below to download the mobilenet-ssd model and convert it using the Intel Model Optimizer.

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --print_all

In [ ]:
# User might have problem to download models from google
#!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o raw_models

#Models are downloaded to /raw_models folder already

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP32 \
-o models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP16 \
-o models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

## Step 2: Running on the IoT DevCloud

### 2.1 Creating job file

Like in the object detection example, we will perform inference on multiple edge compute devices by submitting jobs into the IoT DevCloud queue. To do this, first, we need to create a job file.
The job file is a Bash script, and will be executed on the edge compute node.
For this example, we have written the job file for you in the notebook.
Run the following cell to write this in to the file "store_traffic_manager_job.sh"

In [ ]:
#  -n : number of videos to process

!python3 store_traffic_monitor.py  -d CPU \
                                    -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml \
                                    -l labels.txt \
                               #     -e /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so \
                                    -lp false \
                                    -o results\
                                    -c conf_fp32.txt \
                                    -n 3

**Wait!**

Before moving to step 3, make sure that all the monitor_*  jobs submitted to the queue are completed.

## Step 3: Results

Once the jobs are complete, the queue software writes the stdout and stderr streams into files with names of the form

`monitor_{type}.o{JobID}`

`monitor_{type}.e{JobID}`

(here, the prefix `monitor_{type}` is based on our `-N` argument of `qsub`)

However, for our store monitor example, rather than studying stdout/stderr, you will likely be more interested in viewing the main output in the mp4 videos that are stored in the `results/` directory.
We wrote a short utility script that will display these videos in the notebook. Run the cells below to view the video.
See `demoutils.py` if interested in the script.

Then run the following four cells to see the output  from out jobs.

In [ ]:
videoHTML('Intel® Core CPU',
          ['results/inference_output_Video_0.mp4',
           'results/inference_output_Video_1.mp4', 
           'results/inference_output_Video_2.mp4', 
           'results/Statistics.mp4'],
           'results/stats.txt')